In [1]:
import pandas as pd
import numpy as np
import joblib

In [2]:
proj = pd.read_csv('../../../Documents/GitHub/Frontier-League-Savant/csvs/2024 Pitcher Projections.csv')
proj_hit = pd.read_csv('../../../Documents/GitHub/Frontier-League-Savant/csvs/2024 Hitter Projections.csv')
catchers = pd.read_csv('../../../Documents/GitHub/Frontier-League-Savant/csvs/catcherLeaderboard.csv')
positions = pd.read_csv('../../../Desktop/Joliet Slammers/Baseball Reference/2023 Frontier League Positions.csv')

In [3]:
fl = pd.read_csv('../../../Desktop/Joliet Slammers/AutomatedPitchTagging/AutotaggedCSVs/2023 Frontier League Autotagged (122723).csv')

C:\Users\kellyjc\AppData\Local\Temp\ipykernel_18220\4129827602.py:1: DtypeWarning: Columns (17,74) have mixed types. Specify dtype option on import or set low_memory=False.
  fl = pd.read_csv('../../../Desktop/Joliet Slammers/AutomatedPitchTagging/AutotaggedCSVs/2023 Frontier League Autotagged (122723).csv')


In [4]:
def calculate_park_factors(df):
    pf_list = []
    
    for team in df['HomeTeam'].unique():
        home_data = df[(df['HomeTeam'] == team)]
        away_data = df[(df['AwayTeam'] == team)]
        
        home_xwOBA = home_data['xwOBAcon_model'].mean()
        away_xwOBA = away_data['xwOBAcon_model'].mean()
        
        if away_xwOBA == 0:
            continue
        
        park_factor = home_xwOBA / away_xwOBA
        pf_df = pd.DataFrame({'Team': [team], 'PF': [park_factor]})
        pf_list.append(pf_df)

    pf_df = pd.DataFrame({'Team': ['ESG'], 'PF': [1]})
    pf_list.append(pf_df)
    
    pf_df = pd.concat(pf_list, ignore_index=True)
    mean_pf = pf_df['PF'].mean()
    scaling_factor = 100 / mean_pf
    
    pf_df['PF'] = pf_df['PF'] * scaling_factor
    
    return pf_df

park_factors_df = calculate_park_factors(fl)

In [5]:
proj.Team = proj.Team.replace({'Aigles de Trois-Rivières': 'TRI', 
                             'Evansville Otters': 'EVN',
                             "Florence Y'Alls": 'FLR', 
                             'Gateway Grizzlies': 'GAT',
                             "Joliet Slammers": 'JOL',
                             "Lake Erie Crushers": 'LEC',
                             "New Jersey Jackals": 'NJJ',
                             "New York Boulders": 'RKL',
                             'Ottawa Titans': 'OTT',
                             "Capitales de Québec": 'QUE',
                             "Schaumburg Boomers": 'SCH',
                             'Sussex County Miners': 'SSX',
                             'Tri-City Valley Cats': 'TRC',
                             'Washington WildThings': 'WST',
                             'Windy City ThunderBolts': 'WCT',
                             'Empire State Greys': 'ESG'})

proj_hit.Team = proj_hit.Team.replace({'Aigles de Trois-Rivières': 'TRI', 
                             'Evansville Otters': 'EVN',
                             "Florence Y'Alls": 'FLR', 
                             'Gateway Grizzlies': 'GAT',
                             "Joliet Slammers": 'JOL',
                             "Lake Erie Crushers": 'LEC',
                             "New Jersey Jackals": 'NJJ',
                             "New York Boulders": 'RKL',
                             'Ottawa Titans': 'OTT',
                             "Capitales de Québec": 'QUE',
                             "Schaumburg Boomers": 'SCH',
                             'Sussex County Miners': 'SSX',
                             'Tri-City Valley Cats': 'TRC',
                             'Washington WildThings': 'WST',
                             'Windy City ThunderBolts': 'WCT', 
                             'Empire State Greys': 'ESG'})

park_factors_df.Team = park_factors_df.Team.replace({'Trois Rivieres Aigles': 'TRI', 
                             'Evansville otters': 'EVN',
                             "Florence Y'alls": 'FLR', 
                             'Gateway grizzlies': 'GAT',
                             "Joliet slammers": 'JOL',
                             "Lake Erie Crushers": 'LEC',
                             "New jersey jackals": 'NJJ',
                             "New York Boulders": 'RKL',
                             'Ottawa titans': 'OTT',
                             "Quebec Capitales": 'QUE',
                             "Schaumburg Boomers": 'SCH',
                             'Sussex county miners': 'SSX',
                             'Valleycats': 'TRC',
                             'Washington Wild Things': 'WST',
                             'Windy City ThunderBolts': 'WCT',
                             'Empire State Greys': 'ESG'})

In [6]:
proj = pd.merge(park_factors_df, proj, on='Team', how='right')

In [7]:
proj_hit = pd.merge(park_factors_df, proj_hit, on='Team', how='right')

In [8]:
proj['fipr9'] = (proj['RA9'].mean() - proj['ERA'].mean()) + proj['FIP']
proj['fipr9'] = proj['fipr9'] / (proj['PF'] / 100)
proj['IP/G'] = proj['IP'] / proj['G']
proj['drpw'] = ((((18 - proj['IP/G']) * (proj['fipr9'].mean())) + ((proj['IP/G']) * proj['fipr9'])) / 18 + 2) * 1.5
proj['raap9'] = proj['fipr9'].mean() - proj['fipr9']
proj['wpgaa'] = proj['raap9'] / proj['drpw']
proj['gs/g'] = proj['GS'] / proj['G']
proj['replacement_level'] = (0.03 * (1 - proj['gs/g'])) + (0.12 * proj['gs/g'])
proj['wpgar'] = proj['wpgaa'] + proj['replacement_level']
proj['pit-WAR'] = proj['wpgar'] * (proj['IP'] / 9) + (-0.000682902 * proj['IP'])

In [9]:
proj = proj.dropna(subset=['pit-WAR']).drop(columns=['Unnamed: 0'])

In [10]:
woba_scale = 1.204

woba = proj_hit['wOBA']
league_woba = proj_hit['wOBA'].mean()
league_rpa = (proj_hit['R'] / proj_hit['PA']).mean()
pa = proj_hit['PA']
proj_hit['wRC'] = (((woba - league_woba) / woba_scale) + (league_rpa)) * pa
league_wrcpa = (proj_hit['wRC'] / proj_hit['PA']).mean()
lgR_per_PA = (proj_hit['R'] / proj_hit['PA']).mean() 
proj_hit['wRAA'] = ((proj_hit['wOBA_WAR'] - proj_hit['wOBA_WAR'].mean()) / woba_scale) * proj_hit.PA

In [11]:
wRAA = proj_hit['wRAA']
pf = proj_hit['PF'] / 100

proj_hit['batting_runs'] = proj_hit['wRAA'] + (league_rpa - (pf * league_rpa)) * pa + (league_rpa - (league_wrcpa)) * pa
proj_hit['replacement_level_runs'] = (570 * (1440 / 2430)) * (10 / pa.sum()) * pa

In [12]:
proj_hit['bat-WAR'] = (proj_hit['batting_runs'] + proj_hit['replacement_level_runs']) / 10

In [13]:
wraa_pa = proj_hit['wRAA'] / proj_hit['PA'] 
league_wrcpa = (proj_hit['wRC'] / proj_hit['PA']).mean()
proj_hit['wRC+'] = (((wraa_pa + league_rpa) + (league_rpa - pf * league_rpa)) / (league_wrcpa)) * 100

In [14]:
catchers['cat-WAR'] = catchers['Framing Runs'] / 10

In [15]:
proj_hit = pd.merge(proj_hit, catchers, left_on='Name', right_on='Catcher', how='left')

In [16]:
proj_hit['cat-WAR'].replace('cat-WAR', 0, inplace=True)
proj_hit['cat-WAR'].fillna(0, inplace=True)

In [17]:
proj_hit['bat-WAR'] = proj_hit['bat-WAR'] + proj_hit['cat-WAR']

In [18]:
positions['RV'] = positions['Position'].map({'Centerfielder': 2.5, 'Catcher': 5.5, 'Second': 1.5, 'First': -5, 'Third': 1.5, 'Shortstop': 5, 'Leftfielder': -3.5, 'Rightfielder': -3.5, 'Designated': -8})

In [19]:
proj_hit = pd.merge(proj_hit, positions, on='Name', how='left')

In [20]:
proj_hit['RV'].replace('RV', 0, inplace=True)
proj_hit['RV'].fillna(0, inplace=True)

In [21]:
proj_hit['positional_adj'] = (((proj_hit['PA'] / 4) / 9) / 96) * proj_hit['RV']

In [22]:
proj_hit['bat-WAR'] += proj_hit['positional_adj']

In [24]:
proj_hit[['Name', 'Age', 'PA', '1B', '2B', '3B', 'HR', 'SO', 'SB', 'BB', 'wOBA', 'BA', 'SLG', 'OBP', 'BAbip', 'wRC+', 'bat-WAR']].to_csv('../../../Documents/GitHub/Frontier-League-Savant/csvs/2024 Hitter Projections.csv')

In [26]:
proj[['Name', 'Age', 'IP', 'W', 'L', 'S', 'K', 'BB', 'HR', 'ERA', 'FIP', 'K/9', 'BB/9', 'K/BB', 'pit-WAR']].to_csv('../../../Documents/GitHub/Frontier-League-Savant/csvs/2024 Pitcher Projections.csv')